In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!randam --version

/bin/bash: line 1: randam: command not found


In [ ]:
# We set the backend to TensorFlow. The code works with
# both `tensorflow` and `torch`. It does not work with JAX
# due to the behavior of `jax.numpy.tile` in a jit scope
# (used in `TransformerDecoder.get_causal_attention_mask()`:
# `tile` in JAX does not support a dynamic `reps` argument.
# You can make the code work in JAX by wrapping the
# inside of the `get_causal_attention_mask` method in
# a decorator to prevent jit compilation:
# `with jax.ensure_compile_time_eval():`.
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

import pathlib
import random
import string
import re
import numpy as np
import pandas as pd

import tensorflow as tf_data
import tensorflow.strings as tf_strings

import keras
from keras import layers
from keras import ops
from keras.layers import TextVectorization

In [ ]:
# 歌詞とコードのデータのロード
file_path = '/content/drive/My Drive/lyrics_and_chords_separated.csv'
#file_path = '/content/drive/My Drive/perfect_song_lyrics_chords.csv'
#data = pd.read_csv(file_path, encoding='utf-8')
data = pd.read_csv(file_path, encoding='cp932')

In [ ]:
# CSVファイルのStatusが"1"の行だけを抽出（歌詞とコードが揃っている行）
#filtered_data = data[data['Status'] == 1].dropna(subset=['Lyrics', 'Chords']).reset_index(drop=True)
#filtered_data = data
# データをDataFrameに変換
#df = pd.DataFrame(filtered_data, columns=['URL', 'Lyrics', 'Chords'])
#df = pd.DataFrame(filtered_data, columns=['Lyrics', 'Chords'])
# CSVファイルに保存
#f.to_csv('/content/drive/My Drive/lyrics_and_chords_separated4.csv', index=False, encoding='utf-8')
# 歌詞とコードのペアを確認
data.head()
#filtered_data.head()
#file_path = '/content/drive/My Drive/lyrics_and_chords_separated4.csv'

,URL,Lyrics,Chords
0,https://tabs.ultimate-guitar.com/tab/ed-sheera...,"I found a love for me Darling, just dive right...",G G Em C D G Em C D G Em C G D G Em C D Em C G...
1,https://tabs.ultimate-guitar.com/tab/jeff-buck...,I heard there was a secret chord That David pl...,C Am C Am F G C G C F G Am F G E7 Am F Am F C ...
2,https://tabs.ultimate-guitar.com/tab/elvis-pre...,"Wise men say, only fools rush in But I can't h...",C G Am F C G C C Em Am F C G F G Am F C G C C ...
3,https://tabs.ultimate-guitar.com/tab/john-lege...,What would I do without your smart mouth Drawi...,Em C G D Em Cmaj7 G D Em C G D Em Cmaj7 G D Em...
4,https://tabs.ultimate-guitar.com/tab/jason-mra...,"Well, you done done me in; you bet I felt it I...",G D Em C B G D A E G D Em C G D Em C G D Em C ...


In [ ]:
import csv

text_pairs = []
#with open(file_path, newline='', encoding='utf-8') as csvfile:
with open(file_path, newline='', encoding='cp932') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # ヘッダーをスキップ
    for row in reader:
        if len(row) >= 3:
        #if len(row) >= 2:
            eng = row[1].strip()  # 歌詞
            #eng = row[0].strip()  # 歌詞
            chords = row[2].strip()  # コード進行
            #chords = row[1].strip()  # コード進行
            chords = "[start] " + chords + " [end]"  # 開始・終了トークンを追加
            text_pairs.append((eng, chords))


In [ ]:
for _ in range(5):
    print(random.choice(text_pairs))

("Never mind, I'll find someone like you", '[start] A E F#m [end]')
("And you're married now", '[start] F#m D [end]')
("Come up to meet you, tell you I'm sorry, you don't know how lovely you are", '[start] Bm7 G Gadd9 D Dsus2 [end]')
('Wait for me to come home', '[start] G D [end]')
("When I was six years old I broke my leg And I was running from my brother and his friends And tasted the sweet perfume of the mountain grass as I rolled down I was younger then, take me back to when I Found my heart and broke it here Made friends and lost them through the years And I've not seen the roaring fields in so long, I know I've grown But I can't wait to go home Asus I'm on my way, driving at ninety Asus Down those country lanes singing to Tiny Dancer Asus And I miss the way you make me feel and it's real Asus When we watched the sunset over the castle on the hill Fifteen years old and smoking handrolled cigarettes Running from the law through the backfields and getting drunk with my friends Had 

In [ ]:
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples : num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples :]

print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")
print(f"{len(test_pairs)} test pairs")

1037 total pairs
727 training pairs
155 validation pairs
155 test pairs


In [ ]:
i = 0
for _ in range(6):
    print((test_pairs[i]))
    i = i +1

("By now you should've somehow", '[start] Em7 G G [end]')
('Well me, I fall in love with you every single day', '[start] G Dm G Am [end]')
('And if the night is burning', '[start] Am7 Am7 Em [end]')
('and still those voices are calling from far away', '[start] F F C [end]')
('Raise a glass of wine for the last time', '[start] C Em G D Am7 [end]')
('Darling, just dive right in, and follow my lead', '[start] Em C C D [end]')


In [ ]:
strip_chars = string.punctuation
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")
strip_chars = strip_chars.replace("/", "")#この行がないと"/"が出力されない

vocab_size = 15000
sequence_length = 20
batch_size = 64

def custom_standardization(input_string):
    lowercase = tf_strings.lower(input_string)
    return tf_strings.regex_replace(lowercase, "[%s]" % re.escape(strip_chars), "")#区切り文字を削除する


eng_vectorization = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
chords_vectorization = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)

In [ ]:
train_eng_texts = [pair[0] for pair in train_pairs]
train_chords_texts = [pair[1] for pair in train_pairs]
eng_vectorization.adapt(train_eng_texts)
chords_vectorization.adapt(train_chords_texts)

In [ ]:
def format_dataset(eng, chords):
    eng = eng_vectorization(eng)
    chords = chords_vectorization(chords)
    return (
        {
            "encoder_inputs": eng,
            "decoder_inputs": chords[:, :-1],
        },
        chords[:, 1:],
    )


def make_dataset(pairs):
    eng_texts, chords_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    chords_texts = list(chords_texts)
    dataset = tf_data.data.Dataset.from_tensor_slices((eng_texts, chords_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset)
    return dataset.cache().shuffle(2048).prefetch(16)


In [ ]:
train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [ ]:
for inputs, targets in train_ds.take(1):
    print(f'inputs["encoder_inputs"].shape: {inputs["encoder_inputs"].shape}')
    print(f'inputs["decoder_inputs"].shape: {inputs["decoder_inputs"].shape}')
    print(f"targets.shape: {targets.shape}")

inputs["encoder_inputs"].shape: (64, 20)
inputs["decoder_inputs"].shape: (64, 20)
targets.shape: (64, 20)


In [ ]:
from keras.layers import Layer
from keras.saving import register_keras_serializable
import keras
from keras import layers
import keras.ops as ops

@register_keras_serializable(package="Custom")
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [
                layers.Dense(dense_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, mask=None):
        if mask is not None:
            padding_mask = ops.cast(mask[:, None, :], dtype="int32")
        else:
            padding_mask = None

        attention_output = self.attention(
            query=inputs, value=inputs, key=inputs, attention_mask=padding_mask
        )
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "embed_dim": self.embed_dim,
                "dense_dim": self.dense_dim,
                "num_heads": self.num_heads,
            }
        )
        return config

@register_keras_serializable(package="Custom")
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def call(self, inputs):
        length = ops.shape(inputs)[-1]
        positions = ops.arange(0, length, 1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        if mask is None:
            return None
        else:
            return ops.not_equal(inputs, 0)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "sequence_length": self.sequence_length,
                "vocab_size": self.vocab_size,
                "embed_dim": self.embed_dim,
            }
        )
        return config

@register_keras_serializable(package="Custom")
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [
                layers.Dense(latent_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = ops.cast(mask[:, None, :], dtype="int32")
            padding_mask = ops.minimum(padding_mask, causal_mask)
        else:
            padding_mask = None

        attention_output_1 = self.attention_1(
            query=inputs, value=inputs, key=inputs, attention_mask=causal_mask
        )
        out_1 = self.layernorm_1(inputs + attention_output_1)

        attention_output_2 = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        out_2 = self.layernorm_2(out_1 + attention_output_2)

        proj_output = self.dense_proj(out_2)
        return self.layernorm_3(out_2 + proj_output)

    def get_causal_attention_mask(self, inputs):
        input_shape = ops.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = ops.arange(sequence_length)[:, None]
        j = ops.arange(sequence_length)
        mask = ops.cast(i >= j, dtype="int32")
        mask = ops.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = ops.concatenate(
            [ops.expand_dims(batch_size, -1), ops.convert_to_tensor([1, 1])],
            axis=0,
        )
        return ops.tile(mask, mult)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "embed_dim": self.embed_dim,
                "latent_dim": self.latent_dim,
                "num_heads": self.num_heads,
            }
        )
        return config


In [ ]:
embed_dim = 256
latent_dim = 2048
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads)(x)
encoder = keras.Model(encoder_inputs, encoder_outputs)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
encoded_seq_inputs = keras.Input(shape=(None, embed_dim), name="decoder_state_inputs")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, encoded_seq_inputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
decoder = keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)

decoder_outputs = decoder([decoder_inputs, encoder_outputs])
transformer = keras.Model(
    [encoder_inputs, decoder_inputs], decoder_outputs, name="transformer"
)

In [ ]:
epochs = 40  # This should be at least 40 for convergence
             # accuracyが低かったらepochs数を増やす
transformer.summary()
transformer.compile(
    "rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)
transformer.fit(train_ds, epochs=epochs, validation_data=val_ds)

Model: "transformer"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ encoder_inputs            │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ positional_embedding      │ (None, None, 256)      │      3,845,120 │ encoder_inputs[0][0]   │
│ (PositionalEmbedding)     │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ decoder_inputs            │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ transformer_encoder       │ (None, None, 256)      │      3,155,456 │ positional_embedding[… │
│ (TransformerEncoder)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ functional_3 (Functional) │ (None, None, 15000)    │     12,959,640 │ decoder_inputs[0][0],  │
│                           │                        │                │ transformer_encoder[0… │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 19,960,216 (76.14 MB)

 Trainable params: 19,960,216 (76.14 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 61s 5s/step - accuracy: 0.4063 - loss: 7.5427 - val_accuracy: 0.5216 - val_loss: 4.1906
Epoch 2/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 50s 5s/step - accuracy: 0.5709 - loss: 3.4200 - val_accuracy: 0.5224 - val_loss: 2.4588
Epoch 3/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 85s 5s/step - accuracy: 0.5835 - loss: 2.1545 - val_accuracy: 0.5877 - val_loss: 1.8198
Epoch 4/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 51s 5s/step - accuracy: 0.6277 - loss: 1.6536 - val_accuracy: 0.6067 - val_loss: 1.6252
Epoch 5/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 50s 5s/step - accuracy: 0.6298 - loss: 1.5050 - val_accuracy: 0.6112 - val_loss: 1.5269
Epoch 6/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 50s 5s/step - accuracy: 0.6578 - loss: 1.3354 - val_accuracy: 0.6220 - val_loss: 1.4579
Epoch 7/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 82s 5s/step - accuracy: 0.6472 - loss: 1.3791 - val_accuracy: 0.6078 - val_loss: 1.4802
Epoch 8/40
10/10 ━━━━━━━━━━━━━━━━━━━━ 85s 5s/step - accuracy: 0.6466 - loss: 1.3263 - val_accuracy: 0.5937 - val_loss:

In [ ]:
# モデルを保存
# 保存先ファイルパスを指定（.keras形式）
#transformer.save('/content/drive/My Drive/saved_transformer_model.keras')

In [ ]:
#from keras.models import load_model

# モデルをロード
#loaded_model = load_model('/content/drive/My Drive/saved_transformer_model.keras', custom_objects={"PositionalEmbedding": PositionalEmbedding})

# ロードされたモデルの確認
#loaded_model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:391: UserWarning: `build()` was called on layer 'positional_embedding', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:391: UserWarning: `build()` was called on layer 'transformer_encoder', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:391: UserWarning: `build()` was called on layer 'positional_embedding_1',

Model: "transformer"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ encoder_inputs            │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ positional_embedding      │ (None, None, 256)      │      3,845,120 │ encoder_inputs[0][0]   │
│ (PositionalEmbedding)     │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ decoder_inputs            │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ transformer_encoder       │ (None, None, 256)      │      3,155,456 │ positional_embedding[… │
│ (TransformerEncoder)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ functional_3 (Functional) │ (None, None, 15000)    │     12,959,640 │ decoder_inputs[0][0],  │
│                           │                        │                │ transformer_encoder[0… │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 39,920,434 (152.28 MB)

 Trainable params: 19,960,216 (76.14 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 19,960,218 (76.14 MB)

##コード生成の例(実験では使用していない)

In [ ]:
import pandas as pd
import numpy as np
import random

chords_vocab = chords_vectorization.get_vocabulary()
chords_index_lookup = dict(zip(range(len(chords_vocab)), chords_vocab))
max_decoded_sentence_length = 4#出力するコードの最大の数


def decode_sequence(input_sentence):
    tokenized_input_sentence = eng_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = chords_vectorization([decoded_sentence])[:, :-1]
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])

        # ops.argmax(predictions[0, i, :]) is not a concrete value for jax here
        sampled_token_index = ops.convert_to_numpy(
            ops.argmax(predictions[0, i, :])
        ).item(0)
        sampled_token = chords_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token

        if sampled_token == "[end]":
            break
    return decoded_sentence


test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(1):
    #input_sentence = random.choice(test_eng_texts)
    input_sentence = ("I caught a cold because I kept turning the air conditioning on.")
    translated = decode_sequence(input_sentence)

In [ ]:
print((input_sentence))
print((translated))

I caught a cold because I kept turning the air conditioning on.
[start] em d g [end]


##**英文のコード生成(実験3)**

In [ ]:
chords_vocab = chords_vectorization.get_vocabulary()
chords_index_lookup = dict(zip(range(len(chords_vocab)), chords_vocab))
max_decoded_sentence_length = 4#出力するコードの最大の数


def decode_sequence(input_sentence):
    tokenized_input_sentence = eng_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = chords_vectorization([decoded_sentence])[:, :-1]
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])

        # ops.argmax(predictions[0, i, :]) is not a concrete value for jax here
        sampled_token_index = ops.convert_to_numpy(
            ops.argmax(predictions[0, i, :])
        ).item(0)
        sampled_token = chords_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token

        if sampled_token == "[end]":
            break
    return decoded_sentence.replace("[start] ", "").replace(" [end]", "")


test_eng_texts = [pair[0] for pair in test_pairs]
i=0
file_path = '/content/drive/My Drive/English.csv'
data = pd.read_csv(file_path)
output_path = '/content/drive/My Drive/English_with_Chords.csv'
for _ in range(100):
    #input_sentence = random.choice(test_eng_texts)
    input_sentence = (data['Lyrics'][i])
    translated = decode_sequence(input_sentence)
    print((input_sentence))
    print((translated))
    data.at[i, 'Chords'] = translated
    data.to_csv(output_path, index=False)
    i=i+1

print(f"コード進行を生成し、結果を保存しました: {output_path}")

I caught a cold because I kept turning the air conditioning on. 
em/b bm ab a7sus4
I did not watch the video completely. 
em/b e7/g e7 d
I have a sore throat after discussing passionately about the topic. 
em/b bm g7 dm
A smartphone does not function when the touch ID is not put correctly. 
em/b a7sus4 a7sus4 d
You have to put in the password every time. 
em/b em gadd9
I was exhausted after having club practice after school. 
asus4 fm dsus4 c9
I'd like to have some good ramen tonight.
e7 em d7 gmaj7
I overslept, so I missed the first period.
em/b b7 g d7
Would you come and join our club festival?
c e7 f cmaj7
You might be nervous at first, but I hope you will get accustomed to it sooner or later.
em/b am7 e c/e
The teacher's lessons are always dull, so I am sleepy during class. 
em/b f gmaj7 d7
I have so much homework to write reports to do that I would not complete them.
em/b b7 b7 b7
I made many friends when I attended the welcome party. 
em/b e7 em7 dsus4
Which classes should I take